In [24]:
import pandas as pd
from tqdm import tqdm_notebook
from sklearn.model_selection import train_test_split,KFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from scipy import sparse
import lightgbm as lgb

In [10]:
path = '../data/'
train = pd.read_csv(path+'train_set.csv')
test = pd.read_csv(path+'test_set.csv')
train.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,1,43,management,married,tertiary,no,291,yes,no,unknown,9,may,150,2,-1,0,unknown,0
1,2,42,technician,divorced,primary,no,5076,yes,no,cellular,7,apr,99,1,251,2,other,0
2,3,47,admin.,married,secondary,no,104,yes,yes,cellular,14,jul,77,2,-1,0,unknown,0
3,4,28,management,single,secondary,no,-994,yes,yes,cellular,18,jul,174,2,-1,0,unknown,0
4,5,42,technician,divorced,secondary,no,2974,yes,no,unknown,21,may,187,5,-1,0,unknown,0


In [3]:
test.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,25318,51,housemaid,married,unknown,no,174,no,no,telephone,29,jul,308,3,-1,0,unknown
1,25319,32,management,married,tertiary,no,6059,yes,no,cellular,20,nov,110,2,-1,0,unknown
2,25320,60,retired,married,primary,no,0,no,no,telephone,30,jul,130,3,-1,0,unknown
3,25321,32,student,single,tertiary,no,64,no,no,cellular,30,jun,598,4,105,5,failure
4,25322,41,housemaid,married,secondary,no,0,yes,yes,cellular,15,jul,368,4,-1,0,unknown


In [11]:
test['y'] = -1
cate_features = ['job', 'marital','education','default','housing','loan','contact','month','poutcome']
num_features = ['age', 'balance','day','duration','campaign','pdays','previous']

feature1 = cate_features + num_features
data = pd.concat([train,test])

def feat_count(df,df_feature,fe,value,name=""):
    df_count = pd.DataFrame(df_feature.groupby(fe)[value].count()).reset_index()
    if not name:
        df_count.columns = fe+ [value+"_%s_count" % ("_".join(fe[0]))]
    else:
        df_count.columns = fe+[name]
    df = df.merge(df_count,on=fe,how='left')
    return df,df_count.columns[1]
# for df in feature1:
#     data = feat_count(data,data,['ID'],df)[0]
# data.head()

In [12]:
def feature_count(data,features):
    feature_name = 'count'
    for i in features:
        feature_name += '_'+i
    temp = data.groupby(features).size().reset_index().rename(columns={0:feature_name})
    data = data.merge(temp,'left',on=features)
    return data,feature_name
ll = []
for f in['campaign', 'contact','default','education','housing','job','loan','marital','poutcome','pdays','previous']:
    data,_=feature_count(data,['day','month',f])
    ll.append(_)
sparse_feature= ['campaign','contact','default','education','housing','job','loan','marital','month','poutcome']
dense_feature=['age', 'balance','day','duration','campaign','pdays','previous']

data.head()
    

,ID,age,job,marital,education,default,balance,housing,loan,contact,...,count_day_month_contact,count_day_month_default,count_day_month_education,count_day_month_housing,count_day_month_job,count_day_month_loan,count_day_month_marital,count_day_month_poutcome,count_day_month_pdays,count_day_month_previous
0,1,43,management,married,tertiary,no,291,yes,no,unknown,...,338,326,67,303,46,292,180,338,338,338
1,2,42,technician,divorced,primary,no,5076,yes,no,cellular,...,68,71,9,54,12,62,9,7,1,8
2,3,47,admin.,married,secondary,no,104,yes,yes,cellular,...,204,230,147,138,30,64,166,235,235,235
3,4,28,management,single,secondary,no,-994,yes,yes,cellular,...,194,194,110,130,60,72,53,206,206,206
4,5,42,technician,divorced,secondary,no,2974,yes,no,unknown,...,357,361,213,326,63,313,24,358,358,358


In [13]:
def get_new_columns(name,aggs):
    l = []
    for k in aggs.keys():
        for agg in aggs[k]:
            if str(type(agg)) == "<class 'function'>":
                l.append(name+'_'+k+'_'+'other')
            else:
                l.append(name+'_'+k+'_'+agg)
    return l
for d in tqdm_notebook(sparse_feature):
    aggs = {}
    for s in sparse_feature:
        aggs[s] = ['count','nunique']
    for den in dense_feature:
        aggs[den] = ['mean','max','min','std']
    aggs.pop(d)
    temp = data.groupby(d).agg(aggs).reset_index()
    temp.columns = [d] + get_new_columns(d,aggs)
    data = pd.merge(data,temp,on=d,how='left')

In [14]:
df_train_columns=data.columns.tolist()
print(df_train_columns)

['ID', 'age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y', 'count_day_month_campaign', 'count_day_month_contact', 'count_day_month_default', 'count_day_month_education', 'count_day_month_housing', 'count_day_month_job', 'count_day_month_loan', 'count_day_month_marital', 'count_day_month_poutcome', 'count_day_month_pdays', 'count_day_month_previous', 'campaign_contact_count', 'campaign_contact_nunique', 'campaign_default_count', 'campaign_default_nunique', 'campaign_education_count', 'campaign_education_nunique', 'campaign_housing_count', 'campaign_housing_nunique', 'campaign_job_count', 'campaign_job_nunique', 'campaign_loan_count', 'campaign_loan_nunique', 'campaign_marital_count', 'campaign_marital_nunique', 'campaign_month_count', 'campaign_month_nunique', 'campaign_poutcome_count', 'campaign_poutcome_nunique', 'campaign_age_mean', 'campaign_age_max', 'campaign_age_min

In [15]:
num_features2 = [i for i in data.columns if i not in cate_features if i not in ['ID','y','new_con']]

In [16]:
num_features2

['age',
 'balance',
 'day',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'count_day_month_campaign',
 'count_day_month_contact',
 'count_day_month_default',
 'count_day_month_education',
 'count_day_month_housing',
 'count_day_month_job',
 'count_day_month_loan',
 'count_day_month_marital',
 'count_day_month_poutcome',
 'count_day_month_pdays',
 'count_day_month_previous',
 'campaign_contact_count',
 'campaign_contact_nunique',
 'campaign_default_count',
 'campaign_default_nunique',
 'campaign_education_count',
 'campaign_education_nunique',
 'campaign_housing_count',
 'campaign_housing_nunique',
 'campaign_job_count',
 'campaign_job_nunique',
 'campaign_loan_count',
 'campaign_loan_nunique',
 'campaign_marital_count',
 'campaign_marital_nunique',
 'campaign_month_count',
 'campaign_month_nunique',
 'campaign_poutcome_count',
 'campaign_poutcome_nunique',
 'campaign_age_mean',
 'campaign_age_max',
 'campaign_age_min',
 'campaign_age_std',
 'campaign_balance_mean',
 'campaign_balan

In [20]:
num_features = num_features2

cat_col = [i for i in data.select_dtypes(object).columns if i not in ['ID','y']]
for i in tqdm_notebook(cat_col):
    lbl = LabelEncoder()
    data[i] = lbl.fit_transform(data[i].astype(str))
    
data.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,...,poutcome_duration_min,poutcome_duration_std,poutcome_pdays_mean,poutcome_pdays_max,poutcome_pdays_min,poutcome_pdays_std,poutcome_previous_mean,poutcome_previous_max,poutcome_previous_min,poutcome_previous_std
0,1,43,4,1,2,0,291,1,0,2,...,0,260.008036,-0.952528,528,-1,4.225828,0.000575,7,0,0.052992
1,2,42,9,0,0,0,5076,1,0,0,...,4,241.741093,226.557966,838,1,123.825801,4.019661,275,1,8.374317
2,3,47,0,1,1,0,104,1,1,0,...,0,260.008036,-0.952528,528,-1,4.225828,0.000575,7,0,0.052992
3,4,28,4,2,1,0,-994,1,1,0,...,0,260.008036,-0.952528,528,-1,4.225828,0.000575,7,0,0.052992
4,5,42,9,0,1,0,2974,1,0,2,...,0,260.008036,-0.952528,528,-1,4.225828,0.000575,7,0,0.052992


In [21]:
data['new_con'] = data['job'].astype(str)
for i in ['marital', 'education', 'contact','month','poutcome']:
    data['new_con'] = data['new_con'].astype(str) + '_' + data[i].astype(str)
data['new_con'] = data['new_con'].apply(lambda x: ' '.join(x.split('_')))

In [22]:
data.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,...,poutcome_duration_std,poutcome_pdays_mean,poutcome_pdays_max,poutcome_pdays_min,poutcome_pdays_std,poutcome_previous_mean,poutcome_previous_max,poutcome_previous_min,poutcome_previous_std,new_con
0,1,43,4,1,2,0,291,1,0,2,...,260.008036,-0.952528,528,-1,4.225828,0.000575,7,0,0.052992,4 1 2 2 8 3
1,2,42,9,0,0,0,5076,1,0,0,...,241.741093,226.557966,838,1,123.825801,4.019661,275,1,8.374317,9 0 0 0 0 1
2,3,47,0,1,1,0,104,1,1,0,...,260.008036,-0.952528,528,-1,4.225828,0.000575,7,0,0.052992,0 1 1 0 5 3
3,4,28,4,2,1,0,-994,1,1,0,...,260.008036,-0.952528,528,-1,4.225828,0.000575,7,0,0.052992,4 2 1 0 5 3
4,5,42,9,0,1,0,2974,1,0,2,...,260.008036,-0.952528,528,-1,4.225828,0.000575,7,0,0.052992,9 0 1 2 8 3


In [25]:
vector_feature = ['new_con']


train=data[data.y!=-1]

# train, test, train_y, test_y = train_test_split(train,train_y,test_size=0.2, random_state=2018)
test=data[data.y==-1]
res=test[['ID']]
# test=test.drop(['ID'],axis=1)
train_x=train[num_features]
test_x=test[num_features]
enc = OneHotEncoder()
for feature in cat_col:
    enc.fit(data[feature].values.reshape(-1, 1))
    train_a=enc.transform(train[feature].values.reshape(-1, 1))
    test_a = enc.transform(test[feature].values.reshape(-1, 1))
    train_x= sparse.hstack((train_x, train_a), 'csr')
    test_x = sparse.hstack((test_x, test_a), 'csr')

In [27]:
train_x

<25317x500 sparse matrix of type '<class 'numpy.float64'>'
	with 11317416 stored elements in Compressed Sparse Row format>

In [28]:
cv=CountVectorizer()
for feature in vector_feature:
    cv.fit(data[feature])
    train_a = cv.transform(train[feature])
    test_a = cv.transform(test[feature])
    train_x = sparse.hstack((train_x, train_a), 'csr')
    test_x = sparse.hstack((test_x, test_a), 'csr')

In [29]:
train_x

<25317x502 sparse matrix of type '<class 'numpy.float64'>'
	with 11319012 stored elements in Compressed Sparse Row format>

In [30]:
n_splits=5
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
clf = lgb.LGBMClassifier(
        boosting_type="gbdt", num_leaves=30, reg_alpha=0, reg_lambda=0.,
    max_depth=-1, n_estimators=1500, objective='binary',metric= 'auc',
    subsample=0.95, colsample_bytree=0.7, subsample_freq=1,
    learning_rate=0.02, random_state=2017
    )

train_y=train['y']
res['pred'] = 0
for train_idx, val_idx in kfold.split(train_x):
    clf.random_state = clf.random_state + 1
    train_x1 = train_x[train_idx]
    train_y1 = train_y.loc[train_idx]
    test_x1 = train_x[val_idx]
    test_y1 = train_y.loc[val_idx]
    #,(vali_x,vali_y)
    clf.fit(train_x1, train_y1,eval_set=[(train_x1, train_y1),(test_x1, test_y1)],eval_metric='auc',early_stopping_rounds=100)
    res['pred'] += clf.predict_proba(test_x)[:,1]

res['pred'] = res['pred']/10
# res.to_csv('./basesubmission.csv', index=False)

/Users/sgcx198/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


[1]	training's auc: 0.908369	valid_1's auc: 0.903809
Training until validation scores don't improve for 100 rounds
[2]	training's auc: 0.91096	valid_1's auc: 0.905471
[3]	training's auc: 0.913539	valid_1's auc: 0.908019
[4]	training's auc: 0.926156	valid_1's auc: 0.926665
[5]	training's auc: 0.932818	valid_1's auc: 0.92929
[6]	training's auc: 0.934001	valid_1's auc: 0.92939
[7]	training's auc: 0.937876	valid_1's auc: 0.932513
[8]	training's auc: 0.938627	valid_1's auc: 0.933332
[9]	training's auc: 0.938758	valid_1's auc: 0.932664
[10]	training's auc: 0.939849	valid_1's auc: 0.93329
[11]	training's auc: 0.939314	valid_1's auc: 0.933538
[12]	training's auc: 0.940726	valid_1's auc: 0.934487
[13]	training's auc: 0.941143	valid_1's auc: 0.934772
[14]	training's auc: 0.940989	valid_1's auc: 0.934537
[15]	training's auc: 0.940661	valid_1's auc: 0.934869
[16]	training's auc: 0.941208	valid_1's auc: 0.934699
[17]	training's auc: 0.940926	valid_1's auc: 0.934944
[18]	training's auc: 0.940554	val

[161]	training's auc: 0.959036	valid_1's auc: 0.941628
[162]	training's auc: 0.959132	valid_1's auc: 0.941645
[163]	training's auc: 0.959242	valid_1's auc: 0.941628
[164]	training's auc: 0.95931	valid_1's auc: 0.941528
[165]	training's auc: 0.959419	valid_1's auc: 0.941505
[166]	training's auc: 0.959523	valid_1's auc: 0.941525
[167]	training's auc: 0.959595	valid_1's auc: 0.941434
[168]	training's auc: 0.959713	valid_1's auc: 0.941418
[169]	training's auc: 0.959802	valid_1's auc: 0.941447
[170]	training's auc: 0.959893	valid_1's auc: 0.941471
[171]	training's auc: 0.959986	valid_1's auc: 0.941515
[172]	training's auc: 0.960148	valid_1's auc: 0.941528
[173]	training's auc: 0.960212	valid_1's auc: 0.941536
[174]	training's auc: 0.960277	valid_1's auc: 0.941503
[175]	training's auc: 0.960404	valid_1's auc: 0.941497
[176]	training's auc: 0.960516	valid_1's auc: 0.941452
[177]	training's auc: 0.960595	valid_1's auc: 0.941416
[178]	training's auc: 0.960719	valid_1's auc: 0.941396
[179]	train

/Users/sgcx198/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


[1]	training's auc: 0.920834	valid_1's auc: 0.894923
Training until validation scores don't improve for 100 rounds
[2]	training's auc: 0.923644	valid_1's auc: 0.894877
[3]	training's auc: 0.926231	valid_1's auc: 0.904057
[4]	training's auc: 0.929014	valid_1's auc: 0.907277
[5]	training's auc: 0.93014	valid_1's auc: 0.90794
[6]	training's auc: 0.934252	valid_1's auc: 0.917491
[7]	training's auc: 0.937838	valid_1's auc: 0.923723
[8]	training's auc: 0.938315	valid_1's auc: 0.922971
[9]	training's auc: 0.9387	valid_1's auc: 0.924411
[10]	training's auc: 0.939644	valid_1's auc: 0.924661
[11]	training's auc: 0.94011	valid_1's auc: 0.925633
[12]	training's auc: 0.940323	valid_1's auc: 0.926907
[13]	training's auc: 0.939987	valid_1's auc: 0.928733
[14]	training's auc: 0.940398	valid_1's auc: 0.928407
[15]	training's auc: 0.941144	valid_1's auc: 0.928555
[16]	training's auc: 0.94149	valid_1's auc: 0.928488
[17]	training's auc: 0.942128	valid_1's auc: 0.929077
[18]	training's auc: 0.94235	valid_

...

[184]	training's auc: 0.961676	valid_1's auc: 0.939931
[185]	training's auc: 0.961781	valid_1's auc: 0.939965
[186]	training's auc: 0.961889	valid_1's auc: 0.939941
[187]	training's auc: 0.961986	valid_1's auc: 0.93998
[188]	training's auc: 0.962076	valid_1's auc: 0.939987
[189]	training's auc: 0.962166	valid_1's auc: 0.940014
[190]	training's auc: 0.962264	valid_1's auc: 0.940017
[191]	training's auc: 0.962316	valid_1's auc: 0.940004
[192]	training's auc: 0.962385	valid_1's auc: 0.940008
[193]	training's auc: 0.962473	valid_1's auc: 0.939983
[194]	training's auc: 0.962551	valid_1's auc: 0.940024
[195]	training's auc: 0.962651	valid_1's auc: 0.939985
[196]	training's auc: 0.96273	valid_1's auc: 0.939999
[197]	training's auc: 0.962803	valid_1's auc: 0.939994
[198]	training's auc: 0.962901	valid_1's auc: 0.940017
[199]	training's auc: 0.962967	valid_1's auc: 0.939996
[200]	training's auc: 0.963077	valid_1's auc: 0.940104
[201]	training's auc: 0.963177	valid_1's auc: 0.940115
[202]	traini

[349]	training's auc: 0.973658	valid_1's auc: 0.940735
[350]	training's auc: 0.973731	valid_1's auc: 0.940732
[351]	training's auc: 0.973823	valid_1's auc: 0.940696
[352]	training's auc: 0.973916	valid_1's auc: 0.940719
[353]	training's auc: 0.973959	valid_1's auc: 0.94076
[354]	training's auc: 0.97404	valid_1's auc: 0.940732
[355]	training's auc: 0.974115	valid_1's auc: 0.940743
[356]	training's auc: 0.974168	valid_1's auc: 0.940768
[357]	training's auc: 0.974229	valid_1's auc: 0.940721
[358]	training's auc: 0.974299	valid_1's auc: 0.940725
[359]	training's auc: 0.974367	valid_1's auc: 0.940728
[360]	training's auc: 0.974429	valid_1's auc: 0.940667
[361]	training's auc: 0.974497	valid_1's auc: 0.940689
[362]	training's auc: 0.974553	valid_1's auc: 0.940658
[363]	training's auc: 0.974595	valid_1's auc: 0.94067
[364]	training's auc: 0.974658	valid_1's auc: 0.940595
[365]	training's auc: 0.97471	valid_1's auc: 0.94059
[366]	training's auc: 0.97477	valid_1's auc: 0.940638
[367]	training's

[108]	training's auc: 0.955076	valid_1's auc: 0.927518
[109]	training's auc: 0.955177	valid_1's auc: 0.927577
[110]	training's auc: 0.955312	valid_1's auc: 0.927568
[111]	training's auc: 0.955384	valid_1's auc: 0.927559
[112]	training's auc: 0.955466	valid_1's auc: 0.927461
[113]	training's auc: 0.955561	valid_1's auc: 0.92741
[114]	training's auc: 0.955657	valid_1's auc: 0.927563
[115]	training's auc: 0.955808	valid_1's auc: 0.92769
[116]	training's auc: 0.955921	valid_1's auc: 0.927713
[117]	training's auc: 0.956015	valid_1's auc: 0.927877
[118]	training's auc: 0.956112	valid_1's auc: 0.927937
[119]	training's auc: 0.956203	valid_1's auc: 0.92803
[120]	training's auc: 0.956284	valid_1's auc: 0.928012
[121]	training's auc: 0.956473	valid_1's auc: 0.928108
[122]	training's auc: 0.956559	valid_1's auc: 0.928188
[123]	training's auc: 0.956701	valid_1's auc: 0.928129
[124]	training's auc: 0.95679	valid_1's auc: 0.928127
[125]	training's auc: 0.9569	valid_1's auc: 0.92819
[126]	training's 

[270]	training's auc: 0.969253	valid_1's auc: 0.929408
[271]	training's auc: 0.969321	valid_1's auc: 0.929446
[272]	training's auc: 0.969388	valid_1's auc: 0.929412
[273]	training's auc: 0.969441	valid_1's auc: 0.929457
[274]	training's auc: 0.969518	valid_1's auc: 0.929457
[275]	training's auc: 0.969584	valid_1's auc: 0.929515
[276]	training's auc: 0.969659	valid_1's auc: 0.929496
[277]	training's auc: 0.969708	valid_1's auc: 0.929519
[278]	training's auc: 0.969766	valid_1's auc: 0.929482
[279]	training's auc: 0.969842	valid_1's auc: 0.929449
[280]	training's auc: 0.969922	valid_1's auc: 0.929426
[281]	training's auc: 0.969979	valid_1's auc: 0.92938
[282]	training's auc: 0.970046	valid_1's auc: 0.929419
[283]	training's auc: 0.970097	valid_1's auc: 0.929391
[284]	training's auc: 0.970163	valid_1's auc: 0.929381
[285]	training's auc: 0.970224	valid_1's auc: 0.929421
[286]	training's auc: 0.97029	valid_1's auc: 0.929418
[287]	training's auc: 0.970371	valid_1's auc: 0.929454
[288]	traini

/Users/sgcx198/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
